In [1]:
import numpy as np 
import pandas as pd
import numpy as np
import os
import imutils
import dlib 
import cv2 
import imageio.v2 as imageio
from imutils import face_utils
from skimage.transform import resize
import time

In [2]:
people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
data_types = ['words']
folder_enum = ['01','02','03','04','05','06','07','08', '09', '10']
instances = ['01','02','03','04','05','06','07','08', '09', '10']

words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']          
words_di = {i:words[i] for i in range(len(words))}

In [3]:
max_seq_length = 22

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []


MAX_WIDTH = 100
MAX_HEIGHT = 100

In [4]:
t1 = time.time()
UNSEEN_VALIDATION_SPLIT = ['F07', 'M02']
# UNSEEN_VALIDATION_SPLIT = ['F05']

UNSEEN_TEST_SPLIT = ['F04']

directory = r'cropped' 

for person_id in people:
    tx1 = time.time()
    for data_type in data_types:

        # A for loop that iterates through the different words that are lip read by each person. for count, value in enumerate(values): 
        for word_index, word in enumerate(folder_enum):
#             print(f"Word : '{words[word_index]}'")
            for iteration in instances:
                path = os.path.join(directory, person_id, data_type, word, iteration)
#                 filelist = sorted(os.listdir(path + '/'))
                filelist = sorted(os.listdir(path))
                

                sequence = [] 
                for img_name in filelist:
                    if img_name.startswith('color'):
                        image = imageio.imread(path + '/' + img_name)
                        image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
                        
                        # This is being done to convert a float-valued image (with values between 0 and 1) to an 8-bit grayscale image with values between 0 and 255.
                        image = 255 * image
                        # Convert to integer data type pixels.
                        image = image.astype(np.uint8)
                        sequence.append(image)        

                # add zeros to remaining indeces 
                pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]   #single image placeholder                       
                sequence.extend(pad_array * (max_seq_length - len(sequence)))
                sequence = np.array(sequence)
                
                # assign the sequence to repective splits
                if person_id in UNSEEN_TEST_SPLIT:
                    X_test.append(sequence)
                    y_test.append(word_index)
                elif person_id in UNSEEN_VALIDATION_SPLIT:
                    X_val.append(sequence)
                    y_val.append(word_index)
                else:
                    X_train.append(sequence)
                    y_train.append(word_index)    
    tx2 = time.time()
    print(f'Finished reading images for person {person_id}. Time taken : {tx2 - tx1} secs.')    
    
t2 = time.time()
print(f"Time taken for creating constant size 3D Tensors from those cropped lip regions : {t2 - t1} secs.")
# A tensor is a multi-dimensional array that can store data of any type (integers, floating point values, etc.). In deep learning, tensors are used to store the data that is fed into the network

C:\Users\prana\AppData\Local\Temp\ipykernel_30204\2629557079.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path + '/' + img_name)


Finished reading images for person F01. Time taken : 2.1898748874664307 secs.
Finished reading images for person F02. Time taken : 2.0407867431640625 secs.
Finished reading images for person F04. Time taken : 4.069188356399536 secs.
Finished reading images for person F05. Time taken : 3.6780033111572266 secs.
Finished reading images for person F06. Time taken : 3.664707899093628 secs.
Finished reading images for person F07. Time taken : 3.0289440155029297 secs.
Finished reading images for person F08. Time taken : 2.5199313163757324 secs.
Finished reading images for person F09. Time taken : 2.5148324966430664 secs.
Finished reading images for person F10. Time taken : 1.9978485107421875 secs.
Finished reading images for person F11. Time taken : 2.1335208415985107 secs.
Finished reading images for person M01. Time taken : 2.846742868423462 secs.
Finished reading images for person M02. Time taken : 3.2498836517333984 secs.
Finished reading images for person M04. Time taken : 2.874885082244

In [5]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [6]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1200, 22, 100, 100)
(200, 22, 100, 100)
(100, 22, 100, 100)


In [7]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [8]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(1200,)
(200,)
(100,)


In [9]:
def normalize_it(X):
    
    # keepdims=True keeps same size as input. This is useful for broadcasting the result.
    # Broadcasting is a technique in NumPy where it automatically expands an array 
    # with smaller dimensions to match the shape of an array with larger dimensions during element-wise operations.
    # For example, in this case, if the shape of the filter was (22, 1, 1) instead of (1, 22, 1, 1),
    #  the result of the convolution would not be broadcastable and an error would be raised. By having the shape (1, 22, 1, 1), 
    # it allows the result to be broadcast to (1200, 22, 1, 1), which is the shape of the output volume.
    v_min = X.min(axis=(2, 3), keepdims=True) #op shape (1200, 22, 1, 1). 
    v_max = X.max(axis=(2, 3), keepdims=True)
    X = (X - v_min)/(v_max - v_min)
    X = np.nan_to_num(X)
    return X

In [10]:
from keras.utils import np_utils, generic_utils

In [11]:
X_train = normalize_it(X_train)
X_val = normalize_it(X_val)
X_test = normalize_it(X_test)

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
y_val = np_utils.to_categorical(y_val, 10)

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_val, y_val = shuffle(X_val, y_val, random_state=0)

C:\Users\prana\AppData\Local\Temp\ipykernel_30204\1818949968.py:11: RuntimeWarning: invalid value encountered in divide
  X = (X - v_min)/(v_max - v_min)


In [12]:
X_train = np.expand_dims(X_train, axis=4)
X_val = np.expand_dims(X_val, axis=4)
X_test = np.expand_dims(X_test, axis=4) # op : (100, 22, 100, 100, 1)
# This is often used when working with Convolutional Neural Networks (CNNs) to give them the correct data shape for processing.
# This additional dimension can be interpreted as a channel dimension in case of image data.

In [13]:
np.save('splits/X_train',X_train)
np.save('splits/X_val',X_val)
np.save('splits/X_test',X_test)
np.save('splits/y_train',y_train)
np.save('splits/y_val',y_val)
np.save('splits/y_test',y_test)

In [14]:

splits = r'splits/'
X_train = np.load(splits + 'X_train.npy')
X_val = np.load(splits + 'X_val.npy')
X_test = np.load(splits + 'X_test.npy')
y_train = np.load(splits + 'y_train.npy')
y_val = np.load(splits + 'y_val.npy')
y_test = np.load(splits + 'y_test.npy')

In [15]:
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
from keras.utils.vis_utils import plot_model
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
import matplotlib.pyplot as plt

In [16]:
model = Sequential()

# 1st layer group
model.add(Conv3D(32, (3, 3, 3), strides = 1, input_shape=(22, 100, 100, 1), activation='relu', padding='valid'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(64, (3, 3, 3), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(128, (3, 3, 3), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

# reshape layer
# shape = model.get_output_shape_at(0)
shape = model.layers[-1].output_shape

# he reshaping is necessary for the LSTM layer because LSTM layers in 
# Keras require a 3D input with the shape of (batch_size, timesteps, features)
model.add(Reshape((shape[-1],shape[1]*shape[2]*shape[3])))


# LSTMS - Recurrent Network Layer
# The "return_sequences" means that the output from this LSTM layer will be returned as a sequence.
#  In other words, the output of each time step will be kept and passed on to the next layer.
#  If "return_sequences" was set to False, only the final output of the LSTM layer will be returned.
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(.5))

model.add((Flatten()))

# # FC layers group
model.add(Dense(2048, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.5))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 20, 98, 98, 32)    896       
                                                                 
 max_pooling3d (MaxPooling3D  (None, 10, 49, 49, 32)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 8, 47, 47, 64)     55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 4, 23, 23, 64)    0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 2, 21, 21, 128)    221312    
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 1, 10, 10, 128)   0

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
import visualkeras
from PIL import ImageFont

font = ImageFont.truetype("arial.ttf", 32)  # using comic sans is strictly prohibited!
visualkeras.layered_view(model, legend=True, font=font, spacing=50) 

In [ ]:
t1 = time.time()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=45, batch_size= 16)
t2 = time.time()
print()
print(f"Training time : {t2 - t1} secs.")

model.save('lip_model_cnn_lstm.h5')
model.save_weights('lip_model_cnn_lstm_weights.h5')

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

savedModel = load_model('lip_model_cnn_lstm.h5')

In [18]:
ypred = savedModel.predict(X_test)

4/4 [==============================] - 7s 1s/step


In [19]:
predicted_words = [words[i] for i in np.argmax(ypred, axis=1)]
actual_words = [words[i] for i in np.argmax(y_test, axis=1)] 

In [20]:
correct = 0
for p, a in zip(predicted_words, actual_words):
    if p == a:
        correct += 1
    print(f"Predicted : {p} \t Actual : {a}")

accuracy = correct/len(actual_words)
print(f"Accuracy = {accuracy} on completely unseen data")

Predicted : Hello 	 Actual : Connection
Predicted : Previous 	 Actual : Hello
Predicted : Begin 	 Actual : Begin
Predicted : Navigation 	 Actual : Previous
Predicted : Stop 	 Actual : Stop
Predicted : Stop 	 Actual : Web
Predicted : Choose 	 Actual : Choose
Predicted : Stop 	 Actual : Stop
Predicted : Navigation 	 Actual : Previous
Predicted : Navigation 	 Actual : Web
Predicted : Navigation 	 Actual : Previous
Predicted : Navigation 	 Actual : Web
Predicted : Stop 	 Actual : Stop
Predicted : Choose 	 Actual : Choose
Predicted : Begin 	 Actual : Begin
Predicted : Previous 	 Actual : Navigation
Predicted : Start 	 Actual : Connection
Predicted : Stop 	 Actual : Connection
Predicted : Navigation 	 Actual : Navigation
Predicted : Connection 	 Actual : Begin
Predicted : Navigation 	 Actual : Next
Predicted : Hello 	 Actual : Start
Predicted : Navigation 	 Actual : Begin
Predicted : Connection 	 Actual : Stop
Predicted : Navigation 	 Actual : Next
Predicted : Navigation 	 Actual : Next
Pred